In [2]:
import networkx as nx
from tqdm import tqdm
import random
from leafer import Leafer
import numpy as np
import pickle
import os
import glob

In [3]:
# G = nx.read_edgelist("./IsA_Graph2.edgelist", delimiter=" ", create_using=nx.DiGraph)
G = nx.read_edgelist("test_ru_filter.edgelist", delimiter="\t", create_using=nx.DiGraph)

In [4]:
l = Leafer(G)
# iterator = l.leafs_generator()

In [16]:
train, test = l.split_train_test(
    generation_depth=0,  # до какого уровня в топ. сортировке идти
    p=0.2,  # вероятность что подходящий случай уйдет в тест
    p_divide_leafs=0.5,
    # вероятность что листья поделим пополам трейн-тест
    # а не засунем целый случай в трейн или в тест
    min_to_test_rate=0.3,
    # минимальное количество доли вершин которых не было в
    # трейне чтобы поделить пополам на трейн-тест
    # то есть если 6\10 вершин были трейне то значит все 10 в трейн
    # если 5\10 были в трейне, то значит оставшиеся можем кинуть в тест
    weights=[0.01, 0.49, 0.4, 0.05, 0.05],
    # веса в соответствии
    # один ребенок, только листья, не только листья
    # триплеты с 2 родителями, триплеты такие что мидл нода имеет
    # 1 ребенка
)

In [17]:
len(train), len(test)

(1826, 388)

In [18]:
ls = [elem for elem in test if elem["case"] == "only_leafs_divided"]
len(ls)

36

In [19]:
train_count = {}
for elem in train:
    if elem["case"] in train_count.keys():
        train_count[elem["case"]] += 1

    else:
        train_count[elem["case"]] = 1

test_count = {}
for elem in test:
    if elem["case"] in test_count.keys():
        test_count[elem["case"]] += 1

    else:
        test_count[elem["case"]] = 1

In [20]:
train_count, test_count

({'leafs_and_no_leafs': 199,
  'only_leafs_all': 144,
  'only_leafs_divided': 111,
  'simple_triplet_grandparent': 275,
  'simple_triplet_2parent': 632,
  'only_child_leaf': 465},
 {'simple_triplet_grandparent': 58,
  'only_leafs_divided': 36,
  'leafs_and_no_leafs': 46,
  'only_leafs_all': 26,
  'simple_triplet_2parent': 126,
  'only_child_leaf': 96})

In [33]:
test[:5]

[{'children': ['africación.n.1',
   'dicción.n.1',
   'epéntesis.n.1',
   'corvejón.n.1',
   'nasalización.n.1',
   'sandhi.n.1',
   'síncope.n.1',
   'consonante_vibrante_múltiple.n.1'],
  'parents': 'articulación.n.1',
  'grandparents': None,
  'case': 'leafs_and_no_leafs'},
 {'children': ['nervio_abducens.n.1',
   'nervio_accesorio.n.1',
   'nervio_auditivo.n.1',
   'nervio_facial.n.1',
   'nervio_trigémino.n.1',
   'nervio_olfatorio.n.1'],
  'parents': 'nervio_craneal.n.1',
  'grandparents': None,
  'case': 'only_leafs_divided',
  'brothers': ['nervio_troclear.n.1',
   'nervio_glosofaríngeo.n.1',
   'nervio_hipogloso.n.1',
   'nervio_oculomotor.n.1',
   'nervio_óptico.n.1',
   'nervio_vago.n.1']},
 {'children': ['esclerosis_múltiple.n.1', 'enfermedad_de_Parkinson.n.1'],
  'parents': 'enfermedad_degenerativa.n.1',
  'grandparents': None,
  'case': 'leafs_and_no_leafs'},
 {'children': ['Carissa_macrocarpa.n.1', 'Carissa_bispinosa.n.1'],
  'parents': 'carissa.n.1',
  'grandparents': N

In [34]:
num_leaks = 0
for vertex in l.collector.test_verteces:
    if vertex in l.collector.train_verteces:
        num_leaks += 1

In [23]:
num_leaks

0

In [24]:
num_leaks = 0
for vertex in l.collector.train_verteces:
    if vertex in l.collector.test_verteces:
        num_leaks += 1

In [25]:
num_leaks

0

In [24]:
lang = 'en'
name_train = "../babel_datasets/reweighted_wnet_train_" + lang + "_babel.pickle"
name_test = "../babel_datasets/reweighted_wnet_test_" + lang + "_babel.pickle"

with open(name_train, "wb") as handle:
    pickle.dump(train, handle)

with open(name_test, "wb") as handle:
    pickle.dump(test, handle)

In [41]:
all_data = glob.glob(os.path.join(".", "*isa.edgelist"))
all_data

['./fr_isa.edgelist',
 './it_isa.edgelist',
 './en_isa.edgelist',
 './de_isa.edgelist',
 './es_isa.edgelist',
 './ru_isa.edgelist']

In [3]:
global_train = []
global_test = []

all_data = glob.glob(os.path.join("data/wnet_babel/", "*isa.edgelist"))
for path in tqdm(all_data):
    lang = path.replace("data/wnet_babel/", "").replace("_isa.edgelist", "")

    G = nx.read_edgelist(path, delimiter="\t", create_using=nx.DiGraph)
    l = Leafer(G)
    train, test = l.split_train_test(
    generation_depth=1,  # до какого уровня в топ. сортировке идти
    p=0.05,  # вероятность что подходящий случай уйдет в тест
    p_divide_leafs=0.5,
    # вероятность что листья поделим пополам трейн-тест
    # а не засунем целый случай в трейн или в тест
    min_to_test_rate=0.3,
    # минимальное количество доли вершин которых не было в
    # трейне чтобы поделить пополам на трейн-тест
    # то есть если 6\10 вершин были трейне то значит все 10 в трейн
    # если 5\10 были в трейне, то значит оставшиеся можем кинуть в тест
    weights=[0.01, 0.49, 0.4, 0.05, 0.05],
    # веса в соответствии
    # один ребенок, только листья, не только листья
    # триплеты с 2 родителями, триплеты такие что мидл нода имеет
    # 1 ребенка
)
    random.shuffle(train)
    random.shuffle(test)

    name_train = "../babel_datasets/wnet_only/train_" + lang + "_babel.pickle"
    name_test = "../babel_datasets/wnet_only/test_" + lang + "_babel.pickle"

    with open(name_train, "wb") as handle:
        pickle.dump(train, handle)

    with open(name_test, "wb") as handle:
        pickle.dump(test, handle)

    global_train.extend(train)
    global_test.extend(test)

name_train = "../babel_datasets/wnet_only/global_train_babel.pickle"
name_test = "../babel_datasets/wnet_only/global_test_babel.pickle"


random.shuffle(global_train)
with open(name_train, "wb") as handle:
    pickle.dump(global_train, handle)

with open(name_test, "wb") as handle:
    pickle.dump(global_test, handle)

100%|██████████| 6/6 [00:27<00:00,  4.64s/it]


In [52]:
name_train = "../babel_datasets/wnet_only/global_train_babel.pickle"
name_test = "../babel_datasets/wnet_only/global_test_babel.pickle"

with open(name_train, "wb") as handle:
    pickle.dump(global_train, handle)

with open(name_test, "wb") as handle:
    pickle.dump(global_test, handle)

In [48]:
len(global_test), len(global_train)

(1980, 38808)

In [49]:
max_len = 0
k = 0
for vert in global_train:
    cur_len = len(vert["children"])
    if cur_len > 50:
        k += 1
        max_len = cur_len
        max_i = vert

In [51]:
max_i

{'children': 'conditions_normales_de_température_et_de_pression.n.1',
 'parents': ['atmosphère.n.1', 'température_standard.n.1'],
 'grandparents': None,
 'case': 'simple_triplet_2parent'}

In [17]:
def predict_child_with_parent_and_grandparent(elem):
    """
    hyperhypenym: arthropod.n.01,
    hypernym: insect.n.01, hyponyms:
    (blackly)

    hyperhypenym: elem['grandparents'],
    hypernym: elem['parents'], hyponyms:
    elem['children']

    Fly is a hyponym for the word “insect".
    Predict hyponyms for the word “fly”. Answer:
    """

    # transformed_term = (
    #     "hyperhypenym: "
    #     + ", ".join(elem["grandparents"])
    #     + ", hypernym: "
    #     + elem["parents"]
    #     + ", hyponyms:"
    # )
    transformed_term = (
        ", ".join(elem["grandparents"])
        + " are hyponyms for the word '"
        + elem["parents"]
        + "'. Predict hyponyms for the word '"
        + elem["parents"]
        + "'. Answer:"
    )
    return transformed_term